In [1]:
import pandas as pd
import sqlite3
import gdown
import os.path
import csv

In [3]:
csv_file = 'tennis_data.csv'
if not os.path.isfile(csv_file):
    gdown.download(id='1k-aEbSDXA7SvyPoHD6AjFgfo79dfmkAI', output=csv_file)

In [2]:
df = pd.read_csv(csv_file)

First technique, all in a single table and from there we insert into other tables.

In [4]:
with open(csv_file) as f:
    rdr = csv.reader(f)
    headers = next(rdr)
    rows = []
    for row in rdr:
        rows.append(row)

In [5]:
connection = sqlite3.connect('db')
cursor = connection.cursor()

In [5]:
query = """
INSERT INTO report (ATP, Location, Tournament, Date, Series, Court, Surface, Round, "Best of", Winner, Loser, WRank, LRank, WPts, LPts, W1, L1, W2, L2, W3, L3, W4, L4, W5, L5, Comment, pl1_flag, pl1_year_pro, pl1_weight, pl1_height, pl1_hand, pl2_flag, pl2_year_pro, pl2_weight, pl2_height, pl2_hand)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""
cursor.executemany(query, rows)
connection.commit()

In [6]:
query = """
INSERT INTO players (name, flag, year_pro, weight, height, hand)
SELECT DISTINCT * FROM (
SELECT Winner, pl1_flag, pl1_year_pro, pl1_weight, pl1_height, nullif(pl1_hand, '')
FROM report
UNION
SELECT Loser, pl2_flag, pl2_year_pro, pl2_weight, pl2_height, nullif(pl2_hand, '')
FROM report) t;
"""
cursor.execute(query)
connection.commit()

In [8]:
query = """
INSERT INTO locations (name)
SELECT DISTINCT Location
FROM report;
"""
cursor.execute(query)
connection.commit()

In [7]:
query = """
INSERT INTO tournaments (name)
SELECT DISTINCT Tournament
FROM report;
"""
cursor.execute(query)
connection.commit()

In [7]:
query = """
INSERT INTO events (year, atp, series, tournament_id)
SELECT DISTINCT
    cast(substr(Date, 7, 4) AS INTEGER),
    ATP,
    Series,
    (SELECT id FROM tournaments WHERE name = Tournament)
FROM report;
"""
cursor.execute(query)
connection.commit()

IntegrityError: UNIQUE constraint failed: events.year, events.atp